In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
import pickle
from scipy import spatial, sparse
import numpy as np
from numba import cuda
import time

In [3]:
data = pd.read_csv('~/fires_merged_weather.csv', index_col=0,
                  #dtype for smaller representation
                  dtype={'STAT_CAUSE_DESCR': 'category', 'STATE': 'category', 'DISCOVERY_MONTH': 'category',
                        'Fog': 'bool', 'FunnelCloud': 'bool', 'Hail': 'bool', 'Rain': 'bool',
                        'Snow': 'bool', 'Thunder': 'bool'}
                  )

making doy relative to year, equal to days since 1/1/1992 now

In [4]:
data['DAY'] = (data['FIRE_YEAR']-1992)*365+data['DISCOVERY_DOY']

sort the df

In [5]:
data = data.sort_values(by=['DAY','LATITUDE','LONGITUDE'])

In [6]:
Day_Cts = data.groupby('DAY').agg('count').z

only consider rows within the given time range (need to include lat/lon ranges)

need to use cdist for lat/lon

In [12]:
def simpleA_time(data,day_cts):
    N = data.shape[0]
    A = sparse.lil_matrix((N,N))
    day = data.iloc[0].DAY 
    row_inc=0
    last_day = max(day_cts.index)
    for i in range(N):
        #if the day has changed, reset incrementor
        if data.iloc[i].DAY != day:
            day = data.iloc[i].DAY
            row_inc=0
            #print(day)
        #edge case 1
        if day>1:
            yesterday_index = i-row_inc-day_cts.loc[day-1]
        else:
            yesterday_index = 0
        #edge case 2
        if day<(last_day-1):
            tomorrow_index = i-row_inc+day_cts.loc[day]+day_cts.iloc[day+1]
        else:
            tomorrow_index = N-1
        #loop over possible entries
        for j in range(yesterday_index,tomorrow_index):
            A[i,j] = 1
        #increment day-relative row
        row_inc+=1
    return A

In [13]:
subset = data[0:10000]
Day_Cts = subset.groupby('DAY').agg('count').z
start = time.time()
simpleA_time(subset,Day_Cts)
end = time.time()
print(end-start)

31.236283779144287


In [14]:
print((end-start)*data.shape[0]/10000/60/60)

1.631631621576349
